<a href="https://colab.research.google.com/github/prashantk1220/my-jupyter-notebooks/blob/master/KerasEmbedding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from tensorflow.keras.layers import Input, Embedding, Flatten
from tensorflow.keras.models import Model
from keras.layers import Subtract
import pandas as pd
import numpy as np

In [ ]:
games_df = pd.read_csv('/content/games_data/games_season.csv')
games_df.head()

,season,team_1,team_2,home,score_diff,score_1,score_2,won
0,1985,3745,6664,0,17,81,64,1
1,1985,126,7493,1,7,77,70,1
2,1985,288,3593,1,7,63,56,1
3,1985,1846,9881,1,16,70,54,1
4,1985,2675,10298,1,12,86,74,1


In [ ]:
n_teams = np.unique(games_df[['team_1', 'team_2']]).shape[0]
team_lookup = Embedding(input_dim=n_teams, output_dim=1, input_length=1, name='Team-Strength')

In [ ]:
teamid_in = Input(shape=(1,))
strength_lookup = team_lookup(teamid_in)
strength_lookup_flat = Flatten()(strength_lookup)
team_strength_model = Model(teamid_in, strength_lookup_flat, name='Team-Strength-Model')

---




Creating 2 input layers for for Team 1 and Team 2 and storing their strength values.

In [ ]:
team_in_1 = Input(shape=(1,), name="Team-1-In")
team_in_2 = Input(shape=(1,), name="Team-2-In")

In [ ]:
team_1_strength = team_strength_model(team_in_1)
team_2_strength = team_strength_model(team_in_2)

In [ ]:
score_diff = Subtract()([team_1_strength, team_2_strength])

In [ ]:
model = Model([team_in_1, team_in_2], score_diff)
model.compile(optimizer='adam', loss='mae')

Fit the model on the two teams as input and the score diff:

In [ ]:
input_1 = games_df['team_1']
input_2 = games_df['team_2']
target_out = games_df['score_diff']
model.fit([input_1, input_2], target_out, batch_size=2048, epochs=1, validation_split=0.1, verbose=True)

138/138 [==============================] - 0s 3ms/step - loss: 12.0736 - val_loss: 11.8040


---
Test it on different dataset:



In [ ]:
games_df_2 = pd.read_csv('/content/games_data/games_tourney.csv')
test_team_1 = games_df_2['team_1']
test_team_2 = games_df_2['team_2']
scores = model.evaluate([test_team_1, test_team_2], games_df_2['score_diff'], verbose=True)

133/133 [==============================] - 0s 824us/step - loss: 11.6665
